In [1]:
using ParallelTemperingMonteCarlo,BenchmarkTools,StaticArrays

Looking into the implementation of the EAM. A semi-two-body potential 

In [2]:
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
[0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
[0.1811648830,        0.5575692066,        0.2931316748],
[0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
[0.4742970144,        0.3445967202,       -0.2931309590],
[0.4742970205,       -0.3445967231,       -0.2931309559],
[0.0000007214,       -0.0000000073,       -0.6554619185]]

nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886
bc_cu13 = SphericalBC(radius=8*AtoBohr)
start_config = Config(pos_cu13, bc_cu13)

Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-5.03606052999e-8, -4.52217680244e-8, -3.3916326018300003e-8], [-7.5150356316912e-6, -1.4388744371399999e-8, 6.736624211084434], [1.861953810320738, -5.73051487387157, 3.012712005572341], [-4.874670455235371, -3.541653030973376, 3.01270453061964], [-4.874670517929186, 3.541652881947096, 3.0127044987588487], [1.8619537075439925, 5.730514845094081, 3.0127119541839686], [6.025416511525561, 2.26108840122e-8, 3.0127165935262603], [-1.8619538082652034, -5.73051493450985, -3.012712020988853], [-6.025416612246772, -1.1202665260590001e-7, -3.0127166603311455], [-1.8619539110419487, 5.730514784455802, -3.0127120723772256], [4.87467035451416, 3.5416529415576075, -3.0127045974245252], [4.874670417207975, -3.541652971362864, -3.012704565563734], [7.414314421091401e-6, -7.502702422230001e-8, -6.736624278917086]], SphericalBC{Float64}(228.54811596737582))

Generating the necessary dist2_matrix

In [3]:
dis2mat = [distance2(a,b) for a in pos_cu13, b in pos_cu13]

13×13 Matrix{Float64}:
  0.0      45.3821   45.3821   45.3821  …   45.3821   45.3821   45.3821
 45.3821    0.0      50.1732   50.1732     131.355   131.355   181.528
 45.3821   50.1732    0.0      50.1732     131.355    50.1732  131.355
 45.3821   50.1732   50.1732    0.0        181.528   131.355   131.355
 45.3821   50.1732  131.355    50.1732     131.355   181.528   131.355
 45.3821   50.1732  131.355   131.355   …   50.1732  131.355   131.355
 45.3821   50.1732   50.1732  131.355       50.1732   50.1732  131.355
 45.3821  131.355    50.1732   50.1732     131.355    50.1732   50.1732
 45.3821  131.355   131.355    50.1732     131.355   131.355    50.1732
 45.3821  131.355   181.528   131.355       50.1732  131.355    50.1732
 45.3821  131.355   131.355   181.528   …    0.0      50.1732   50.1732
 45.3821  131.355    50.1732  131.355       50.1732    0.0      50.1732
 45.3821  181.528   131.355   131.355       50.1732   50.1732    0.0

The mathematical form of the EAM is:

$$
E = \frac{1}{2}\sum_{i,j :i \neq j}\epsilon\left(\frac{a}{r_{ij}}\right)^n - \epsilon C \sum_{i}\left[\sum_{j\neq i} \left(\frac{a}{r_{ij}}\right)^m \right]^{1/2}
$$

We have two terms, one similar to the LJ repulsion term, one a density term $F(\rho _i)$.

We can consider that $\frac{\epsilon a^n}{2}$ and $\epsilon C a^{m/2} $ can be taken out of the sum. Thus a single sum represents contracting a vector in the dist matrix. 

In [4]:
struct EmbeddedAtomPotential <: AbstractDimerPotential
    n::Float64
    m::Float64
    ean::Float64
    eCam::Float64
end
function EmbeddedAtomPotential(n,m,ϵ,C,a)
    epsan = ϵ * a^n / 2
    epsCam = ϵ * C * a^(m/2)
    return EmbeddedAtomPotential(n,m,epsan,epsCam)
end

EmbeddedAtomPotential

In [5]:
evtohartree = 0.0367493

n = 8.482
m = 4.692
ϵ = evtohartree*0.370
a = 0.25*nmtobohr
C = 27.561


copperpot = EmbeddedAtomPotential(n,m,ϵ,C,a)
suttonchenpot = EmbeddedAtomPotential(9.0,6.0,0.0126*evtohartree,39.432,0.3612*nmtobohr)
copperpot2 = EmbeddedAtomPotential(9.05 , 5.005 , 0.0225*evtohartree,33.17,0.327*nmtobohr)

EmbeddedAtomPotential(9.05, 5.005, 5949.446479587037, 2.6153101861477057)

In [6]:
copperpot

EmbeddedAtomPotential(8.482, 4.692, 3565.9126256755612, 14.313533476465633)

In [7]:
invrexp(r2,expterm) = ifelse(r2 != 0 , 1/sqrt(r2)^expterm, 0. )
calcvec(distvec,expterm) = sum(invrexp.(distvec,Ref(expterm)))
function energyatomi(distvec,pot::EmbeddedAtomPotential) 
    Fterm = pot.eCam*sqrt(calcvec(distvec,pot.m))
    phiterm = pot.ean*calcvec(distvec,pot.n)
    return phiterm - Fterm
end


energyatomi (generic function with 1 method)

In [8]:
E_vec = [energyatomi(distrow,copperpot) for distrow in eachrow(dis2mat)]
sum(E_vec)

-5.090193312823571

In [9]:
E_vec2 = [energyatomi(distrow,copperpot2) for distrow in eachrow(dis2mat)]
sum(E_vec2)

-0.6734688354904295

# An alternative approach

Considering that the $\rho_i$ term depends on the square of sums, we can't simply update the total energy by considering the change to a single $r_{ij}$ on the global energy.

Rather than calculating $E_i$ and being unable to _separate_ the terms, we can store the $\phi_i,\rho_i$ terms. This may result in slower calculation of the global terms, but it may speed up the calculation of $E_{new}$ at each step.